In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined bty the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from __future__ import print_function

import os
import json
import re
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import time, random, math
import re  #regular expression
import operator
from collections import defaultdict, Counter, OrderedDict
import nltk #natural language processing
import six
import scipy.sparse as sp
from operator import itemgetter

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
#matplotlib.rc('figure', figsize = (14, 7))
matplotlib.rc('font', size = 14)
# Remove grid lines
# Set backgound color to white
matplotlib.rc('axes', facecolor = 'white')

import seaborn as sns
sns.set(style="whitegrid")

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from sklearn.ensemble import RandomForestClassifier as RFC, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.metrics import accuracy_score, log_loss

print(os.listdir("../input"))

In [ ]:
def timer(func):
    def wrapper(*args, **kwargs):
        ts = time.time()
        results = func(*args, **kwargs)
        te = time.time()
        print("Time to execute {} = {} seconds".format(func.__name__, te - ts))
        return results
              
    return wrapper

In [ ]:
#default configurations
kwargs = dict(histtype='stepfilled', alpha=1.0, bins=15)

def createRandomColorCode():
    r = np.random.uniform(0, 1)
    g = np.random.uniform(0, 1)
    b = np.random.uniform(0, 1)
    return tuple((r, g, b))

def histogram(*args, **kwargs):
    ax = args[0]
    data = args[1]
    title = args[2]
    x_label = args[3] if len(args) > 3 else ""
    y_label = args[4] if len(args) > 4 else ""
    ax.hist(data, **kwargs)
    ax.set_title(title)
    
def barplot(ax, data, title, ticklabels, horizontal=False):
    """
    Plots a horizontal or vertical barplot
    Parameters
    -------------
    ax - the figure axes
    """
    N = len(data)
    ind = np.arange(N)
    width = 0.75
    colors = ['#33FFFF', '#79E5DB', '#A2A6DF', '#F2B8F0', '#DDEA53']
    if horizontal:
        rects = ax.barh(ind, data, width, color=colors)
        ax.set_yticks(ind)
        ax.set_yticklabels(ticklabels)
        ax.set_title(title)
        for i, v in enumerate(rects):
            ax.text(data[i], i + 0.25, str(data[i]), weight='bold', color='b')
    else:
        rects = ax.bar(ind, data, width, color=colors)
        ax.set_xticks(ind)
        ax.set_xticklabels(ticklabels)
        for tick in ax.get_xticklabels():
            tick.set_rotation(90)
            
        for i, rect in enumerate(rects):
            xloc = rect.get_x() + 0.02
            yloc = rect.get_height()
            ax.text(xloc, yloc, str(data[i]), weight='bold', color='b')
            
def plotBarFromDicts(results, metrics, colors):
    fig = plt.figure(figsize=(10, 3))
    N = len(metrics)

    for i, item in enumerate(metrics):
        ax = fig.add_subplot(1, N, i + 1)
        data = [item[metrics[i]] for item in results]
        barplot(ax, data, metrics[i].capitalize(), [item['label'] for item in results])
    
    fig.tight_layout()
    plt.show()
    plt.gcf().clear()
    
def plotNGramDistribution(cuisines, metrics):
    # Create the plot object
    fig, axarr = plt.subplots(len(cuisines), 3, sharex=False, sharey=True, figsize=(18,60), squeeze=False)
    for i, cuisine in enumerate(cuisines):
        for j, metric in enumerate(metrics):
            data = gb.get_group(cuisine)[metric]
            title = '{}-{}'.format(cuisine, metric.split('_')[0])
            histogram(axarr[i][j], data, title, **kwargs)
            
    plt.tight_layout()
    plt.show()
    plt.gcf().clear()
    
@timer
def fit_transform(data):
    train_vectors = vectorizer.fit_transform(data)
    return train_vectors


In [ ]:
def formatNum(num):
    return float("{0:.2f}".format(num))

@timer
def plot_most_used_ingredient():
    top_ingredients = []
    labels = []
    indices = []
    counts = gb.apply(len)
    
    for i, name in enumerate(list(gb.groups)):
        word_freq = create_vocabulary(gb.get_group(name)['ingredients'])
        word_freq = sorted(word_freq.items(), key=lambda x: x[1])
        
        _ = word_freq[-1][1]/counts.loc[name]
        _ = math.ceil(_*100)
        top_ingredients.append(_)
        labels.append(name + ' (' + word_freq[-1][0] + ') ')
        indices.append(name)
        
    return pd.DataFrame({'label': labels, 'values': top_ingredients}, index=indices)

def find_variants(df, pattern):
    pat = re.compile(r'\w+\s+' + pattern, re.IGNORECASE)
    matches = [pat.findall(",".join(row)) for row in list(df['ingredients'])]
    return list(set(np.hstack(matches)))

@timer
def getVariants(df, ingredients):
    _ = {}
    for ingredient in ingredients:
        _[ingredient] = find_variants(df, ingredient)
        print('Processing {} completed'.format(ingredient))
    return _

@timer
def getIngredientVariants(pattern):
    results = pd.Series(name='variants')
    for i, name in enumerate(list(gb.groups)):
        frame = gb.get_group(name)
        results.loc[name] = len(find_variants(frame, pattern))
    return results

def get_percentage_ingredient_in_cuisine(column):
    data = train.groupby('cuisine').agg({
        column: sum
    })
    data['total'] = train.groupby('cuisine').apply(len).values
    data['%'] = data.apply(lambda x: float("{0:.2f}".format(x['has_cheese'] / x['total'])), axis=1)
    return data

def preprocess(doc):
    doc = doc.lower()
    doc = re.sub(r'yoghurt', 'yogurt', doc)
    doc = re.sub(r"\bchees\b", "cheese", doc)
    doc = re.sub(r"ic\s*cream", "ice cream", doc)
    doc = re.sub(r"5 spice[s]?", "panch phoron", doc)
    doc = re.sub(r"low sodium salt", "kosher salt", doc)
    return doc

def get_copy(df):
    temp = df.copy()
    temp['joined_ingredients'] = temp['ingredients'].apply(lambda row: ", ".join(row))
    return temp

In [ ]:
class AyataVectorizer(object):
    def __init__(self, max_df=1.0, min_df=1, max_features=None, ngram_range=(1, 1), lowercase=True, token_pattern=r"(?u)\b\w[\w\s]+\w\b", stop_words=None):
        self.stop_words = stop_words
        self.vocabulary = None
        self.max_df = max_df
        self.min_df = min_df
        self.max_features = max_features
        self.lowercase = lowercase
        self.token_pattern = re.compile(token_pattern)
        self.ngram_range = ngram_range
        
    def build_preprocessor(self):
        return lambda doc: doc.lower()
    
    def build_tokenizer(self):
        return lambda x: self.token_pattern.findall(x)
    
    def build_analyzer(self):
        preprocess = self.build_preprocessor()
        tokenize = self.build_tokenizer()
        return lambda doc: tokenize(preprocess(doc))
    
    def word_ngrams(self, tokens):
        if self.stop_words is not None:
            tokens = [w for w in tokens if w not in self.stop_words]
        
        min_n, max_n = self.ngram_range
        if max_n != 1:
            if min_n == 1:
                # copy all unigrams when max_n > 1 and min_n == 1
                original_tokens = tokens[:]
                min_n += 1
            else:
                # dont need unigrams
                tokens = []
                
            n_original_tokens = len(original_tokens)
            tokens_append = tokens.append
            space_join = " ".join

            for n in range(min_n, min(max_n + 1, n_original_tokens + 1)):
                for i in range(n_original_tokens - n + 1):
                    tokens_append(space_join(original_tokens[i: i + n]))
        return tokens
        
    def create_term_frequency_matrix(self, docs):
        j_indices = []
        values = []
        # initialize to zero
        indptr = [0]
        
        analyze = self.build_analyzer()
        
        vocab = defaultdict()
        vocab.default_factory = vocab.__len__
        for i, doc in enumerate(docs):
            feature_counter = {}
            for token in self.word_ngrams(analyze(doc)):
                feature_idx = vocab[token]
                if feature_idx not in feature_counter:
                    feature_counter[feature_idx] = 1
                else:
                    feature_counter[feature_idx] += 1
            
            j_indices.extend(feature_counter.keys())
            values.extend(feature_counter.values())
            indptr.append(len(j_indices))
        
        j_indices = np.asarray(j_indices, dtype=np.int32)
        indptr = np.asarray(indptr, dtype=np.int32)
        values = np.asarray(values, dtype=np.int32)
        
        X = sp.csr_matrix((values, j_indices, indptr), shape=(len(indptr) - 1, len(vocab)), dtype=int)
        X.sort_indices()
        return dict(vocab), X
    
    def sort_features(self, X, vocabulary):
        sorted_vocab = sorted(vocabulary.items(), key=lambda x: x[0]) #sorted(six.iteritems(vocabulary))
        map_index = np.empty(len(sorted_vocab), dtype=np.int32)
        for new_indx, (item, old_index) in enumerate(sorted_vocab):
            vocabulary[item] = new_indx
            map_index[old_index] = new_indx
            
        X.indices = map_index.take(X.indices)
        return X
    
    def document_frequency(self, X):
        return np.bincount(X.indices)
    
    def limit_features(self, X, vocabulary, max_df, min_df):
        dfs = self.document_frequency(X)
        tfs = np.asarray(X.sum(axis=0)).ravel()
        mask = np.ones(len(dfs), dtype=bool)
        mask &= dfs <= max_df
        mask &= dfs >= min_df
        
        #feature selection
        mask_inds = (-tfs[mask]).argsort()
        if self.max_features is None:
            mask_inds = mask_inds[:]
        else:
            mask_inds = mask_inds[:self.max_features]
        
        #map old to new indices
        new_mask = np.zeros(len(dfs), dtype=bool)
        new_mask[np.where(mask)[0][mask_inds]] = True
        mask = new_mask
        
        new_indices = np.cumsum(mask) - 1
        removed_terms = set()
        for term, old_index in list(six.iteritems(vocabulary)):
            if mask[old_index]:
                vocabulary[term] = new_indices[old_index]
            else:
                del vocabulary[term]
                removed_terms.add(term)
                
        kept_indices = np.where(mask)[0]
        return X[:, kept_indices], removed_terms
        
    def fit(self, docs):
        vocabulary, X = self.create_term_frequency_matrix(docs)
        X = self.sort_features(X, vocabulary)
        n_doc = X.shape[0]
        min_doc_count = self.min_df
        max_doc_count = self.max_df * n_doc
        X, stop_words = self.limit_features(X, vocabulary, max_doc_count, min_doc_count)
        self.stop_words = stop_words
        self.vocabulary = vocabulary
        return X
    
    def get_feature_names(self):
        return [t for t, i in sorted(six.iteritems(self.vocabulary),key=itemgetter(1))]
        

In [ ]:
NEGAT_STR = r'\bno[\s-][a-zA-Z-\s]+'
NEGAT_PATTERN = re.compile(NEGAT_STR, re.IGNORECASE)

HYPHEN_STR = r'^[a-zA-Z-\s]*[-][a-zA-Z-\s]+\b'
HYPHEN_PAT = re.compile(HYPHEN_STR)

In [ ]:
with open("../input/train.json", 'r') as file:
    data = json.load(file)

In [ ]:
df = pd.DataFrame(data)
train = df.copy(deep=True)

In [ ]:
gb = train.groupby('cuisine')
print('There are {} unique groups'.format(len(gb.groups)))

In [ ]:
train['joined_ingredients'] = train['ingredients'].apply(lambda row: ", ".join(row))

In [ ]:
train['joined_ingredients'] = train['joined_ingredients'].apply(lambda x: preprocess(x))

In [ ]:
train['joined_ingredients'].str.extract(r'([\w\s]*!)').dropna()

In [ ]:
def replaceNumber(doc):
    
    doc = re.sub(r'\d{1,}', 'NUM', doc)
    return doc

In [ ]:
train['joined_ingredients'] = train.apply(lambda x: replaceNumber(x['joined_ingredients']), axis=1)

In [ ]:
train['joined_ingredients'].str.extract(r'(NUM)').dropna().shape

In [ ]:
train['joined_ingredients'] = train['joined_ingredients'].apply(lambda x: preprocess(x))

### Hyphen separated ingredients ###

In [ ]:
train['hyphen_ingrd'] = train.apply(lambda x: HYPHEN_PAT.findall(x['joined_ingredients']), axis=1)

In [ ]:
hyphen_separated_arr = list(np.hstack(train['hyphen_ingrd']))

In [ ]:
_ = defaultdict()
_.default_factory = _.__len__

for index, item in enumerate(hyphen_separated_arr):
    _[item] += 1
    
_ = dict(_)
_ = sorted(_.items(), key=lambda x: x[0])

In [ ]:
d = np.asarray([i[1] for i in _])
mask = np.ones(len(d), dtype=bool)
mask_names = np.asarray([i[0] for i in _])

In [ ]:
#fig, ax = plt.subplots(figsize=(11, 30))
#barplot(ax, d[mask], "", mask_names.take(np.where(mask)[0]), horizontal=True)

In [ ]:
# train['negations'] = train['joined_ingredients'].apply(lambda x: NEGAT_PATTERN.findall(x))

In [ ]:
corpus = train['joined_ingredients']

In [ ]:
vectorizer = CountVectorizer(min_df=2)
data_train = fit_transform(corpus)

In [ ]:
my_vectorizer = AyataVectorizer(max_df=0.9)
X = my_vectorizer.fit(corpus)
train_vectorizer = pd.DataFrame(X.toarray(), columns=my_vectorizer.get_feature_names())

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_train, train.cuisine, test_size=0.33, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
clf = RFC(random_state=42)
clf = clf.fit(X_train, y_train)   

In [ ]:
predictions = clf.predict(X_test)
predict_proba = clf.predict_proba(X_test)
predictions_df = pd.DataFrame({'actuals': y_test, 'predicteds': predictions})

In [ ]:
print('Accuracy: ', accuracy_score(predictions_df['actuals'], predictions_df['predicteds']))
print('Log loss: ', log_loss(predictions_df['actuals'], predict_proba))

### TEST ###

In [ ]:
# with open("../input/test.json", 'r') as file:
#     test = json.load(file)

In [ ]:
# test_data = pd.DataFrame(test)
# test_data[:1]

In [ ]:
# test_data['joined_ingredients'] = test_data['ingredients'].apply(lambda x: ",".join(x))

In [ ]:
# feature_vectors = vectorizer.transform(test_data['joined_ingredients'])

In [ ]:
# vect_df = pd.DataFrame(feature_vectors.toarray(), columns=[vectorizer.get_feature_names()])

In [ ]:
# bestModel = clf
# ids = test_data['id']
# predictions = bestModel.predict(vect_df)

In [ ]:
# submission = pd.DataFrame({"cuisine": predictions, "id": ids})

In [ ]:
# submission.to_csv('submission.csv', index=False)

In [ ]:
# submission['cuisine'].value_counts()

In [ ]:
def createVocab(docs):
    j_indices = []
    values = []
    indptr = [0]
    
    vocab = defaultdict()
    vocab.default_factory = vocab.__len__
    
    for idx, items in enumerate(docs):
        feature_counter = {}
        for n in items:
            feature_idx = vocab[n]
            if feature_idx not in feature_counter:
                feature_counter[feature_idx] = 1
            else:
                feature_counter[feature_idx] += 1
            
        j_indices.extend(feature_counter.keys())
        values.extend(feature_counter.values())
        indptr.append(len(j_indices))
    
    X = sp.csr_matrix((np.asarray(values, dtype=np.int32), np.asarray(j_indices, dtype=np.int32), indptr), shape=(len(indptr) - 1, len(vocab)), dtype=int)
    X.sort_indices()
    return X, dict(vocab)

def sort_features(X, vocabulary):
    sorted_features = sorted(six.iteritems(vocabulary)) #sorted(occurrences.items(), key=lambda x: x[0])
    map_index = np.empty(len(sorted_features), dtype=np.int32)
    sorted_vocab = defaultdict()
    
    for new_val, (term, old_val) in enumerate(sorted_features):
        vocabulary[term] = new_val
        map_index[old_val] = new_val
        
    X.indices = map_index.take(X.indices, mode='clip')
    return X

In [ ]:
corpus1 = [
    'This is the first document.',
    'This document is the second document',
    'And this is the third one.',
    'Is this the first document?',
    ]

In [ ]:
corpus1 = [doc.split() for doc in corpus1]
corpus1

In [ ]:
X_test, vocab_test = createVocab(corpus1)

In [ ]:
X_test.toarray()

In [ ]:
print(vocab_test)

In [ ]:
X_test.indices

In [ ]:
sorted_features = sorted(six.iteritems(vocab_test))
map_index = np.empty(len(sorted_features), dtype=np.int32)
print(sorted_features)

In [ ]:
for new_val, (term, old_val) in enumerate(sorted_features):
    vocab_test[term] = new_val
    map_index[old_val] = new_val

In [ ]:
print(vocab_test)
print()
print(map_index)

In [ ]:
X_test.toarray()

In [ ]:
X_test.indices = map_index.take(X_test.indices)

In [ ]:
X_test.toarray()

In [ ]:
X_test.indices

In [ ]:
sorted(X_test.indices)

In [ ]:
# Document frequency of each term index
dfs = np.bincount(X_test.indices) # on sorted array as in vocabulary
dfs

In [ ]:
print(sorted(vocab_test.items(), key=lambda x: x[0]))

In [ ]:
tfs = np.asarray(X_test.sum(axis=0)).ravel()
tfs

In [ ]:
# placeholder for old indices
mask = np.ones(len(dfs), dtype=bool)
mask

In [ ]:
dfs

In [ ]:
#Find terms occurring in at most n documents
mask &= dfs <= 2
print(dfs)
print()
print(mask)

In [ ]:
tfs

In [ ]:
-tfs[mask]

In [ ]:
mask_inds = (-tfs[mask]).argsort()
mask_inds = mask_inds[:]
mask_inds

In [ ]:
new_mask = np.zeros(len(dfs), dtype=bool)
new_mask

In [ ]:
mask

In [ ]:
np.where(mask)[0]

In [ ]:
np.where(mask)[0][mask_inds]

In [ ]:
# Finds index of kept features
new_mask[np.where(mask)[0][mask_inds]] = True
new_mask 

In [ ]:
mask = new_mask
mask

In [ ]:
np.cumsum(mask)

In [ ]:
new_indices = np.cumsum(mask) - 1 # maps old indices to new
new_indices

In [ ]:
print(vocab_test)

In [ ]:
removed_terms = set()
for term, old_index in list(six.iteritems(vocab_test)):
    if mask[old_index]:
        vocab_test[term] = new_indices[old_index]
    else:
        del vocab_test[term]
        removed_terms.add(term)

In [ ]:
print(vocab_test)

In [ ]:
print(removed_terms)

In [ ]:
kept_indices = np.where(mask)[0]
kept_indices

In [ ]:
X = X_test[:, kept_indices]

In [ ]:
print(sorted(six.iteritems(vocab_test), key=itemgetter(1)))

In [ ]:
df = np.bincount(X.indices)

In [ ]:
n_samples, n_features = X.shape

In [ ]:
idf = np.log(n_samples / df) + 1

In [ ]:
X.toarray()

In [ ]:
sorted(X.indices)

In [ ]:
df

In [ ]:
idf

In [ ]:
_idf_diag = sp.diags(idf, offsets=0,shape=(n_features, n_features), format='csr', dtype=np.float64)

In [ ]:
pd.DataFrame(X.toarray())

In [ ]:
pd.DataFrame(_idf_diag.toarray())

In [ ]:
pd.DataFrame(X * _idf_diag.toarray())